Initializing Spark Context

In [1]:
from pyspark.sql import SparkSession


# jars = ["/home/jovyan/openlineage/libs/openlineage-spark-1.5.0.jar"]
marquez_url = "http://k8s-marquez-marquez-1f83396143-232926722.us-east-1.elb.amazonaws.com/api/v1"
marquez_namespace = 'notebook_experiments'


spark = (SparkSession.builder.master('local')
            .appName('complex_data_types_experiments')
            # .config('spark.jars', ",".join(jars))
            # .config('spark.jars.packages', 'io.openlineage:openlineage-spark:1.5.0')
            # .config('spark.extraListeners', 'io.openlineage.spark.agent.OpenLineageSparkListener')
            # .config('spark.openlineage.appName', 'complex_data_type') # overwriting Spark app name in events 
            
            # # using HTTP transport, sending events directly to Marquez endpoint
            # .config('spark.openlineage.transport.type', 'http')
            # .config('spark.openlineage.transport.url', marquez_url)
            
            # # evolve to capture an env var
            # .config('spark.openlineage.namespace', marquez_namespace)
            .getOrCreate())

spark.sparkContext.setLogLevel("INFO")

Generic DataFrame for initial experiment

In [45]:
generic_df = spark.createDataFrame([
    {'a': 1, 'b': [1,2,3], 'c': {'id': 1, 'name': {'first': 'luis', 'last': 'yamada'}}, 'd': 1.500, 'e': 'blablabla'},
    {'a': 2, 'b': [3,4,5], 'c': {'id': 2, 'name': {'first': 'marcos', 'last': 'andrade'}}, 'd': 1.500, 'e': 'blablabla'},
    {'a': 3, 'b': [6,7,8], 'c': {'id': 3, 'name': {'first': 'walace', 'last': 'morais'}}, 'd': 1.500, 'e': 'blablabla'}
])
generic_df.show(3,False)

+---+---------+-----------------------+---+---------+
|a  |b        |c                      |d  |e        |
+---+---------+-----------------------+---+---------+
|1  |[1, 2, 3]|{name -> null, id -> 1}|1.5|blablabla|
|2  |[3, 4, 5]|{name -> null, id -> 2}|1.5|blablabla|
|3  |[6, 7, 8]|{name -> null, id -> 3}|1.5|blablabla|
+---+---------+-----------------------+---+---------+



In [46]:
generic_df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- c: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- d: double (nullable = true)
 |-- e: string (nullable = true)



In [47]:
for column in generic_df.schema:
    # print(column)
    print(column.name)
    # if column.dataType.simpleString()[:3] == "map":
    #     print("MAP")
    # else:
    print(column.dataType.simpleString().upper())
    # if column.dataType

    
    print("")

a
BIGINT

b
ARRAY<BIGINT>

c
MAP<STRING,BIGINT>

d
DOUBLE

e
STRING



Time to write the dataframe and check Marquez

In [13]:
generic_df.write.mode("overwrite").parquet("./complex_data_type/")